# WaterpyBal API Example
Here we are going develop an example to demonstrate the usage of WaterpyBal library.

## Importing Waterpybal

In [1]:
from waterpybal import dataset_gen,variable_management,SWR,Infiltration,Urban_cycle,PET,Balance, post_process
import os
import netCDF4 as nc

## Dataset Generation - Defining the dimensions

In [2]:
# creating a dataset class
dataset_cl=dataset_gen()

#definig the dataset dimensions

#defining the source that marks the study area.
single_point=True
lat_lon_type=None

#the raster path
sam_raster_dir=None


#defining the time source type
time_type="time_dic"
time_dic={
        "start":["2018","01","01","00"],
        "end":["2022","01","01","00"],
        }
#Define the time interval of the dataset
preferred_date_interval="Daily" 

dataset_cl.ds_dimensions(

            lat_lon_type=None,
            lat_lon_source=None,
            time_source=None,
            time_type=time_type,
            preferred_date_interval=preferred_date_interval,
            lat_name=None,
            lon_name=None,
            time_name=None,
            border_res_dic=None,
            time_dic=time_dic,
            single_point=single_point   
        )



## Dataset Generation - Create the dataset and the variables

In [3]:
#dataset path
dir=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\v6_just_urban\v6_just_urban_single_point.nc"
#define the extra variables that have to be included in the dataset
ds_values_dic={"tmean":"C","tmin":"C","tmax":"C"} 

#If the study area contains an urban zone
urban_ds=True
#Generate the dataset
ds=dataset_cl.var_generation(dir=dir,ds_values_dic=ds_values_dic,urban_ds=urban_ds)

## Variable interpolation and addition to the dataset

In [4]:
ras_sample_dir=sam_raster_dir
csv_dir_temps=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\METEO_9stations.csv"
csv_dir_prec=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\Prec_Val_daily_from_monthly.csv"
time_csv_col="time"
lat_csv_col="lat"
lon_csv_col="lon"

#define the .csv paths
csv_dirs=[csv_dir_prec,csv_dir_temps,csv_dir_temps,csv_dir_temps]
#define if the values have to be distributed through the database (refer to manual)
multiply=[True,False,False,False]

#define the interpolation method
#method="linear: radius=-1:nodata=-9999"

#define the variable names
var_names=["Prec","tmean","tmin","tmax"]

#define the .CSV time intervals
interpolation_time_ints=["Daily","Monthly","Monthly","Monthly"]

#interpolate the variables
for var_name,interpolation_time_int, csv_dir,mult in zip(var_names,interpolation_time_ints,csv_dirs,multiply):
    ds=variable_management.var_introduction_from_csv(ds,csv_dir,var_name,interpolation_time_int,time_csv_col="time",multiply=mult)
    #ds=variable_management.var_interpolation(ds,ras_sample_dir,csv_dir,var_name,method,interpolation_time_int,time_csv_col,lat_csv_col,lon_csv_col,mult)

time ['2018-01-01T12' '2018-01-02T12' '2018-01-03T12' ... '2021-12-29T12'
 '2021-12-30T12' '2021-12-31T12']
dev_by 1
date_dt as type 2018-01
date_dt as type 2018-02
date_dt as type 2018-03
date_dt as type 2018-04
date_dt as type 2018-05
date_dt as type 2018-06
date_dt as type 2018-07
date_dt as type 2018-08
date_dt as type 2018-09
date_dt as type 2018-10
date_dt as type 2018-11
date_dt as type 2018-12
date_dt as type 2019-01
date_dt as type 2019-02
date_dt as type 2019-03
date_dt as type 2019-04
date_dt as type 2019-05
date_dt as type 2019-06
date_dt as type 2019-07
date_dt as type 2019-08
date_dt as type 2019-09
date_dt as type 2019-10
date_dt as type 2019-11
date_dt as type 2019-12
date_dt as type 2020-01
date_dt as type 2020-02
date_dt as type 2020-03
date_dt as type 2020-04
date_dt as type 2020-05
date_dt as type 2020-06
date_dt as type 2020-07
date_dt as type 2020-08
date_dt as type 2020-09
date_dt as type 2020-10
date_dt as type 2020-11
date_dt as type 2020-12
date_dt as type 202

## Soil Water Reserve Calculation


In [5]:
#how the SWR function behaves based on the dataset (refer to the manual)
time_steps=None #same soil properties for the whole dataset. variable space.

# Path to the multiband raster that contains "cc", "pwp" & "rrt" values. 
#raster_SWR_dir=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\modelito_rasters_pixel5\just_urban_cc1_pwp2_rrt3_lu4_hsg5_hc6.tif"
ds["pwp"][:,:,:]=0.12
ds["cc"][:,:,:]=0.23
ds["rrt"][:,:,:]=0.2
#raster bands
#raster_bands_dic_or_val=dict()
#raster_bands_dic_or_val['cc']=1
#raster_bands_dic_or_val['pwp']=2
#raster_bands_dic_or_val['rrt']=3
ds=SWR.swr(ds,time_steps)
#ds=SWR.swr(ds,time_steps,raster_SWR_dir,raster_bands_dic_or_val)

## PET Calculation

In [6]:
pet_method="hargreaves"

#latitude=45 degrees, the temperature values of the database have to be used to calculate the etp
ds=PET.pet(ds,method=pet_method,lat=45,tmean="ds",tmin="ds",tmax="ds")

## Infiltration Calculation

In [7]:
#curve number table path
#CN_table_dir=r"C:\Users\Ash kan\Documents\watbalpy\waterpybal\curve_number_standard_table.xls"

#Path to the multi band raster containing the CN table identifier info
#raster_dir=raster_SWR_dir
#raster bands
#LU_band,HSG_band,ELEV_or_HC_band=4,5,6

#If the Antecedent Moisture Condition (AMC) corrections have to be applied
corrected_cn=True
ds=Infiltration.inf(ds,CN_table_dir=None,
                    raster_dir=None,
                    HSG_band=None,
                    LU_band=None,
                    ELEV_or_HC_band=None,
                    corrected_cn=False,
                    single_cn_val=True,
                    cn_val=81)
#ds=Infiltration.inf(ds,CN_table_dir,raster_dir,HSG_band,LU_band,ELEV_or_HC_band,corrected_cn=corrected_cn)


inf in runoff inf cal 0 0
inf in runoff inf cal 0 after 1394
inf in runoff inf cal 9999 2 0
inf in runoff inf cal 0 2 1394


## Urban Cycle Calculation

In [8]:
#A raster that defines the Urban sites of the study area. other zones have to be nodata (-9999)
#urban_rasters_folder=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\modelito_rasters_pixel5\urban_rasters"
#urban_area_raster_dir=os.path.join(urban_rasters_folder,"DI"+".tif")

#Urban cycle calculation variables (constant values in this example; could be rasters ir from the dataset)
variables_dic={
    'wat_cons': {'input_var':8,'dataset_raster_dir_or_value':"Constant"},
    "wat_net_loss":{'input_var':7,'dataset_raster_dir_or_value':"Constant"},
    "urb_dir_evap":{'input_var':14,'dataset_raster_dir_or_value':"Constant"},
    "urb_indir_evap":{'input_var':4,'dataset_raster_dir_or_value':"Constant"},
    "sew_net_loss_low":{'input_var':3,'dataset_raster_dir_or_value':"Constant"},
    "sew_net_loss_high":{'input_var':8,'dataset_raster_dir_or_value':"Constant"},
    "prec_sewage_threshold":{'input_var':15,'dataset_raster_dir_or_value':"Constant"},
    "runoff_to_sewage":{'input_var':35,'dataset_raster_dir_or_value':"Constant"},
    "dir_infil":{'input_var':25,'dataset_raster_dir_or_value':"Constant"},
    "wat_supp_wells":{'input_var':2.15,'dataset_raster_dir_or_value':"Constant"},
    "wat_supp_wells_loss":{'input_var':6,'dataset_raster_dir_or_value':"Constant"},
    "wat_other":{'input_var':6.3,'dataset_raster_dir_or_value':"Constant"},
    "urban_to_ds_inf_PET_ratio":{'input_var':100,'dataset_raster_dir_or_value':"Constant"}
    }


#ds=Urban_cycle.urban_cycle(ds,urban_area_raster_dir,variables_dic) 
ds=Urban_cycle.urban_cycle(ds,urban_area_raster_dir=None,variables_dic=variables_dic) 

In [9]:
#dir=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\v6_just_urban\v6_just_urban_1.nc"

#ds=nc.Dataset(dir,'r+',format='NETCDF4')


## Balance Calculation

In [10]:
#Initial soil water reserve percentage
init_swr=100

ds=Balance.balance(ds,init_swr=init_swr)


In [11]:
ds.close()

## Visualization and outputs

### Generate a report of the water balance in the study area

In [12]:
#path to the dataset
ds_dir=dir=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\v6_just_urban\v6_just_urban_single_point.nc"

#path to save the results
save_dir=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\v6_just_urban"
#list of the variables to include in the report
var_name_list=["Prec","Irrig","INF","PET","SWR","ETR","Def","Rec","ASWR","URB_INF","URB_EP","Runoff","CN","tmean","tmax","tmin"]
time_dic={
        "start":["2018","01","01","00"],
        "end":["2022","01","01","00"],
        }
#path to the sample raster of the dataset
#sam_raster_dir=sam_raster_dir
#a raster that marks different regions of the study area and the report will create the results based on the are
#identifier_raster_dir=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\modelito_rasters_pixel5\regions.tif"
#import rasterio as rs
#ras=rs.open(identifier_raster_dir)
#identifier_raster_array=ras.read(1)
#region=["1","2","3","4","5","6"]
lat_name,lon_name="lat","lon"
post_process.gen_report(ds_dir,time_dic,var_name_list,save_dir,sam_raster_dir,lat_name,lon_name)


start 2018-01-01T00
end 2022-01-01T00
Unable to calculate the study area. Eash pixel area is assumed as 1 m2


### Create Maps and Raster in specific time intervals

In [ ]:
#path to the dataset
ds_dir=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\v5_modelito_change_zone_5_6_soil_charc\v5_8_3_2023.nc"
#path to save the results
save_dir=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\v5_modelito_change_zone_5_6_soil_charc"
#the range to create the figures or rasters
time_dic={
        "start":["2019","05","26","00"],
        "end":["2019","06","04","00"],
        }
var_name="tmin"
#fig_csv_raster 'Figure', 'Raster' or 'CSV'. To determine the type of the output.
fig_csv_raster="Figure"
post_process.raster_fig_csv(ds_dir,save_dir,time_dic,var_name,fig_csv_raster)

### Create figures or .csv datasheets in a coordination

In [ ]:
#path to the dataset
ds_dir=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\v5_modelito_change_zone_5_6_soil_charc\v5_6_3_2023.nc"

#path to save the results
save_dir=r"C:\Users\Ash kan\Documents\watbalpy\waterball_test\modelito_for_paper_tests\v5_modelito_change_zone_5_6_soil_charc"

#the range to create the figures or rasters
'''time_dic={
        "start":["YYYY","MM","DD","HH"],
        "end":["YYYY","MM","DD","HH"],
        }'''

#the latitude and longitude of the point of interest
lat_val,lon_val= 1,2

#figure or a csv archive? 'Figure', or 'CSV'
to_fig_or_csv='Figure'

post_process.point_fig_csv(ds_dir,save_dir,time_dic,lat_val,lon_val,var_name_list,to_fig_or_csv)